In [ ]:
# Libraries

import matplotlib
import os
import platform
import shutil
import sys

In [ ]:
# Ensure source path

ROOT = os.getcwd()

while not ROOT.endswith('upolanc-thesis') :
	ROOT = os.path.abspath(os.path.join(ROOT, os.pardir))

	if len(ROOT) < len('upolanc-thesis') :
		if   platform.system() == 'Linux'   : ROOT = '/d/hpc/projects/FRI/up4472/upolanc-thesis'
		elif platform.system() == 'Windows' : ROOT = 'C:\\Developer\\Workspace\\PyCharm\\Projects\\upolanc-thesis'
		else : raise ValueError()

		print(f'Warning : could not find correct directory, using default : {ROOT}')
		print()

		break

if ROOT not in sys.path :
	sys.path.append(ROOT)

os.chdir(ROOT)

In [ ]:
# Code

from source.python              import runtime
from source.python.data.feature import feature_anndata
from source.python.data.feature import feature_tsne
from source.python.io           import loader

runtime.set_numpy_format()
runtime.set_pandas_format()
runtime.set_plot_theme()

# 1. Setup

In [ ]:
# Setup some directory paths

FILTER_ID = 2
SUBFOLDER = 'filter' + str(FILTER_ID)

CWD = ROOT
OUT = os.path.join(CWD, 'output')
RES = os.path.join(CWD, 'resources')

OUT_DATA  = os.path.join(OUT,      'nbp03-tsne',    SUBFOLDER)
OUT_PLOT  = os.path.join(OUT_DATA, 'plot')
RES_NBP01 = os.path.join(OUT,      'nbp01-filter',  SUBFOLDER)
RES_NBP02 = os.path.join(OUT,      'nbp02-anndata', SUBFOLDER)

shutil.rmtree(OUT_DATA, ignore_errors = True)

os.makedirs(OUT_DATA, exist_ok = True)
os.makedirs(OUT_PLOT, exist_ok = True)

print(f'     Root Directory : {CWD}')
print(f'   Output Directory : {OUT_DATA}')
print(f'   Output Directory : {OUT_PLOT}')
print(f' Resource Directory : {RES_NBP01}')
print(f' Resource Directory : {RES_NBP02}')
print()

In [ ]:
# Load the annotated data

anndata = loader.load_h5ad(
	filename = os.path.join(RES_NBP02, 'arabidopsis-r36.h5ad')
)

filter_dict = loader.load_json(
	filename = os.path.join(RES_NBP01, 'filter.json')
)

In [ ]:
# Filtered transcripts

keep_transcript = filter_dict['data']['keep_transcript']
drop_transcript = filter_dict['data']['drop_transcript']

# 2. TSNE

## 2.1 Filtering

In [ ]:
# Select only the needed transcripts

keep_anndata = anndata[:, keep_transcript].copy()
drop_anndata = anndata[:, drop_transcript].copy()

In [ ]:
# Select the most prominent genes (not implemented)

# with warnings.catch_warnings() :
# 	warnings.simplefilter('ignore')

# 	mask = feature_tsne.select_genes(
# 		matrix    = keep_anndata.X,
# 		n         = 3000,
# 		threshold = 0,
# 		decay     = 1,
# 		xoffset   = 5,
# 		yoffset   = 0.02
# 	)

# 	keep_anndata = keep_anndata[:, mask].copy()

## 2.2 Features

In [ ]:
# Compute and save the pca from standardized log1p values and keep the top 50 components

keep_anndata = feature_anndata.compute_pca(
	data       = keep_anndata,
	layer      = 'standard',
	store_into = 'pca50',
	components = 50
)

## 2.3 Embeddings

In [ ]:
# Compute and save the tsne embeddings

keep_anndata = feature_tsne.compute_tsne(
	data         = keep_anndata,
	features     = 'pca50',
	store_into   = 'tsne',
	perplexities = [50, 200]
)

## 2.4 Visualization

In [ ]:
# Display tsne visualization for diffrent gene groups

for group in ['tissue', 'age', 'group', 'perturbation'] :
	feature_tsne.visualize(
		data     = keep_anndata,
		feature  = 'tsne',
		groupby  = group.capitalize(),
		filename = os.path.join(OUT_PLOT, 'tsne-' + group),
		alpha    = 0.5,
		size     = 5,
		prop     = {
			'size' : 24
		}
	)

	matplotlib.pyplot.show()